In [ ]:
import os
from config_file import config
import sys
sys.path.append("/export/home/nidebroux/sct_custom")
sys.path.append("/export/home/nidebroux/modules")
os.environ["CUDA_VISIBLE_DEVICES"] = '/gpu:0'
import numpy as np
import pandas as pd
import pickle
import random
from spinalcordtoolbox.image import Image
from sklearn.utils import shuffle
import tkinter
import matplotlib
#comment the line below if necessary
matplotlib.use('TkAgg')
import keras
import tensorflow as tf
from spinalcordtoolbox.deepseg_sc.cnn_models_3d import load_trained_model
from generator import get_training_and_validation_generators
from utils import fetch_data_files, visualize_data, normalize_data, load_3Dpatches, train_model, get_callbacks
from keras import backend as K
import matplotlib.pyplot as plt

main_dir = config["main_dir"]

### IMPORT PARAMETERS FROM CONFIG FILE

In [ ]:
# config['data_dict'] = pickle file containing a dictionary with at least the following keys: subject and contrast_foldname
# This dict is load as a panda dataframe and used by the function utils.fetch_data_files
# IMPORTANT NOTE: the testing dataset is not included in this dataframe
DATA_PD = pd.read_pickle(config['data_dict'])

DATA_FOLD = config["data_dir"]  # where the preprocess data are stored
#MODEL_FOLD = config["path2save"]  # where to store the trained models

MEAN_TRAIN_T2, STD_TRAIN_T2 = 871.309, 557.916  # Mean and SD of the training dataset of the original paper

### CONVERT INPUT IMAGES INTO AN HDF5 FILE

In [ ]:
len_train = int(0.8 * len(DATA_PD.index)) # 80% of the dataset is used for the training, 20% for validation
idx_train = random.sample(range(len(DATA_PD.index)), len_train)
idx_valid = [ii for ii in range(len(DATA_PD.index)) if ii not in idx_train] # the remaining images are used for the validation

In [ ]:
training_files = fetch_data_files(data_frame=DATA_PD[DATA_PD.index.isin(idx_train)],
                                  data_fold=DATA_FOLD,
                                  im_suffixe='_norm',
                                  target_suffixe='_crop_SEG')
validation_files = fetch_data_files(data_frame=DATA_PD[DATA_PD.index.isin(idx_valid)],
                                  data_fold=DATA_FOLD,
                                  im_suffixe='_norm',
                                  target_suffixe='_crop_SEG')

print(training_files)
print(validation_files)

### EXTRACT 3D PATCHES

In [ ]:
# The extracted patches are stored as pickle files (one for training, one for validation).
# If these files already exist, we load them directly (i.e. do not re run the patch extraction).
pkl_train_fname = DATA_FOLD + 'train_data.pkl'
print(pkl_train_fname)
if not os.path.isfile(pkl_train_fname):
    
    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=config["patch_overlap"]) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)
    
    with open(pkl_train_fname, 'wb') as fp:
        pickle.dump(np.array([X_train, y_train]), fp)
else:
    with open (pkl_train_fname, 'rb') as fp:
        X_train, y_train = pickle.load(fp)

In [ ]:

pkl_valid_fname = DATA_FOLD + 'valid_data.pkl'
print(pkl_valid_fname)

if not os.path.isfile(pkl_valid_fname):
    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    with open(pkl_valid_fname, 'wb') as fp:
        pickle.dump(np.array([X_valid, y_valid]), fp)
else:
    with open (pkl_valid_fname, 'rb') as fp:
        X_valid, y_valid = pickle.load(fp)

In [ ]:
print('Number of Training patches:\n\t' + str(X_train.shape[0]))
print('Number of Validation patches:\n\t' + str(X_valid.shape[0]))

### LOAD TRAINED MODEL

In [ ]:
model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')
loss =model.loss
optimizer_class_name = model.optimizer.__class__.__name__
optimizer_config = model.optimizer.get_config()
model.compile(optimizer = optimizer_class_name, loss=loss)

### GET TRAINING AND VALIDATION GENERATORS

In [ ]:
train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=config["batch_size"],
                                                    augment=True,
                                                    augment_flip=True)

print(train_generator,nb_train_steps)

In [ ]:
validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=1,
                                                    augment=False,
                                                    augment_flip=False)
print(validation_generator,nb_valid_steps)

### MAIN FINE-TUNING PIPELINE

The fine-tuning technique and parameters chosen here are based on the results obtained trough the tests below. It depends directly from the training set and should thus be change in function of the used training set.

In [ ]:
for layer in model.layers[:7]:
    layer.trainable = False

K.set_value(model.optimizer.learning_rate,0.00001)

model.compile(optimizer = optimizer_class_name, loss=loss)

history = train_model(model=model,
    path2save=config["path2save"],
    model_name=config["model_name"],
    training_generator=train_generator,
    validation_generator=validation_generator,
    steps_per_epoch=nb_train_steps,
    validation_steps=nb_valid_steps,
    n_epochs=2*config["n_epochs_1"],
    learning_rate_drop=config["learning_rate_drop"],
    learning_rate_patience=config["learning_rate_patience"]
   )

np.save(main_dir + "parameters_analysis/best_results.npy",history.history['val_loss'])

# PARAMETER TESTS

The rest of the script contains a serie of tests made on the following parameters : depth of the network surgery in case of transfer learning, amount of frozen parameters during unfreezing phase, batch size, overlap size, learning rate (for transfer learning and for unfreezing phase) and data augmentation.

### NETWORK SURGERY 


We test here which layers we should replace in a network surgery.


In [ ]:
#surgery for contraction path 
model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
contr_1 = keras.layers.Conv3D(24,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'contr_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[0].output)
contr_2 = keras.layers.BatchNormalization(name = 'contr_batch_1', axis = 1)(contr_1)
contr_3 = keras.layers.Activation('relu',name = 'contr_activation_1')(contr_2)
contr_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'contr_conv3D_2',padding = 'same',kernel_initializer = init)(contr_3)
contr_5 = keras.layers.BatchNormalization(name = 'contr_batch_2',axis=1)(contr_4)
for i in range(6,24):
    if i ==6 :
        contr_6 = model.layers[i](contr_5)
    elif i == 7:
        l= model.layers[i](contr_6)
    elif i == 15 :
        l = model.layers[i]([contr_6,l])
    else :
        l= model.layers[i](l)
    
contr_model = keras.models.Model(model.input,l,name = "contraction_model")

for layer in contr_model.layers[6:24]:
    layer.trainable = False
    
contr_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(contr_model.optimizer.learning_rate,0.001)




In [ ]:
#surgery for expansion path
model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')


init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
long_2 = keras.layers.BatchNormalization(name = 'long_batch_1',axis=1)(long_1)
long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
long_5 = keras.layers.BatchNormalization(name = 'long_batch_2',axis=1)(long_4)
long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

for layer in long_model.layers[:-9]:
    layer.trainable = False
    
long_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(long_model.optimizer.learning_rate,0.001)


In [ ]:
#surgery with both expansion and contraction

model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
contr_1 = keras.layers.Conv3D(24,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'contr_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[0].output)
contr_2 = keras.layers.BatchNormalization(name = 'contr_batch_1', axis = 1)(contr_1)
contr_3 = keras.layers.Activation('relu',name = 'contr_activation_1')(contr_2)
contr_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'contr_conv3D_2',padding = 'same',kernel_initializer = init)(contr_3)
contr_5 = keras.layers.BatchNormalization(name = 'contr_batch_2',axis=1)(contr_4)
for i in range(6,16):
    if i ==6 :
        contr_6 = model.layers[i](contr_5)
    elif i == 7:
        l= model.layers[i](contr_6)
    elif i == 15 :
        l = model.layers[i]([contr_6,l])
    else :
        l= model.layers[i](l)
        
long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(l)
long_2 = keras.layers.BatchNormalization(name = 'long_batch_1',axis=1)(long_1)
long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
long_5 = keras.layers.BatchNormalization(name = 'long_batch_2',axis=1)(long_4)
long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)
    
double_model = keras.models.Model(model.input,long_output,name = "contraction_and_expansion_model")

for layer in double_model.layers[6:16]:
    layer.trainable = False
    
double_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(double_model.optimizer.learning_rate,0.001)

In [ ]:
#surgery of the bottleneck
model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

for i in range(14,24):
    if i ==14 :
        l = model.layers[i](bottle_6)
    elif i == 15 :
        l = model.layers[i]([model.layers[6].output,l])
    else :
        l= model.layers[i](l)
        
bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

i = 0
for layer in bottle_model.layers:
    if i > 7 and i < 14:
        layer.trainable = True
    else:
        layer.trainable = False
    i = i+1
    
bottle_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(bottle_model.optimizer.learning_rate,0.001)


In [ ]:
#fit the expansion model
long_history = train_model(model=long_model,
            path2save=config["path2save_transferlearning"],
            model_name="expansion_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )
np.save(main_dir + "parameters_analysis/expansion_surgery.npy",long_history.history['val_loss'])

In [ ]:
#fit the contraction model
contr_history = train_model(model=contr_model,
            path2save=config["path2save_transferlearning"],
            model_name="contr_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )
np.save(main_dir + "parameters_analysis/contr_surgery.npy",contr_history.history['val_loss'])

In [ ]:
#fit the double model
double_history = train_model(model=double_model,
            path2save=config["path2save_transferlearning"],
            model_name="double_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=config["learning_rate_patience"]
           )
np.save(main_dir + "parameters_analysis/double_surgery.npy",double_history.history['val_loss'])

In [ ]:
#fit the bottle model
bottle_history = train_model(model=bottle_model,
            path2save=config["path2save_transferlearning"],
            model_name="bottle_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=config["learning_rate_patience"]
           )
np.save(main_dir + "parameters_analysis/bottle_surgery.npy",bottle_history.history['val_loss'])

In [ ]:
#clone of the model without training
model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

for layer in model.layers:
    layer.trainable = True

clone_model = tf.keras.models.clone_model(model)

   
clone_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(clone_model.optimizer.learning_rate,0.001)

In [ ]:
clone_history = train_model(model=clone_model,
            path2save=config["path2save_transferlearning"],
            model_name="total_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=config["learning_rate_patience"]
           )
np.save(main_dir + "parameters_analysis/clone.npy",clone_history.history['val_loss'])

In [ ]:
long_surgery = np.load(main_dir+"parameters_analysis/expansion_surgery.npy")
contr_surgery = np.load(main_dir+"parameters_analysis/contr_surgery.npy")
double_surgery = np.load(main_dir+"parameters_analysis/double_surgery.npy")
bottle_surgery = np.load(main_dir+"parameters_analysis/bottle_surgery.npy")
clone = np.load(main_dir+"parameters_analysis/clone.npy")

epochs = range(25,101,5)
mean_val_loss = np.zeros((5,16))

for i in range(16):
    mean_val_loss[0,i]=np.abs(np.mean(long_surgery[20+5*i:20+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(contr_surgery[20+5*i:20+5*i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(double_surgery[20+5*i:20+5*i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(bottle_surgery[20+5*i:20+5*i+5]))
    mean_val_loss[4,i]=np.abs(np.mean(clone[20+5*i:20+5*i+5]))


plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'val_loss for expansion surgery')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'val_loss for contraction surgery')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'val_loss for contraction and expansion surgery')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'val_loss for bottleneck surgery')
plt.plot(epochs,mean_val_loss[4,:],'o--',label = 'val_loss for total replacement')
plt.title('Comparison between different network surgeries ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### BATCH SIZE TEST

In [ ]:
batch_size_train = [1,5,10,20,50]
val_loss_values = np.zeros((5,50))

In [ ]:

for i in range(5):

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=batch_size_train[i],
                                                    augment=True,
                                                    augment_flip=True)


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=1,
                                                    augment=False,
                                                    augment_flip=False)

    model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
    bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
    bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
    bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
    bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
    bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

    for i in range(14,24):
        if i ==14 :
            l = model.layers[i](bottle_6)
        elif i == 15 :
            l = model.layers[i]([model.layers[6].output,l])
        else :
            l= model.layers[i](l)

    bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

    i = 0
    for layer in bottle_model.layers:
        if i > 7 and i < 14:
            layer.trainable = True
        else:
            layer.trainable = False
        i = i+1

    bottle_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(bottle_model.optimizer.learning_rate,0.001)


    #fit the bottle model
    history = train_model(model=bottle_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["batch_size_test",str(nb_train_steps)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=50,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_values[i,:] = history.history['val_loss']
    
np.save(main_dir + "parameters_analysis/batch_size.npy",val_loss_values)

In [ ]:
epochs = range(20,51,5)
val_loss_values = np.load(main_dir + "parameters_analysis/batch_size.npy")
mean_val_loss = np.zeros((5,7))
for i in range(7):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_values[0,15+5*i:15+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_values[1,15+5*i:15+5*i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_values[2,15+5*i:15+5*i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_values[3,15+5*i:15+5*i+5]))
    mean_val_loss[4,i]=np.abs(np.mean(val_loss_values[4,15+5*i:15+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'batch size = 1 - iteration = 271')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'batch size = 5 - iteration = 54')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'batch size = 10 - iteration = 27')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'batch size = 20 - iteration = 13')
plt.plot(epochs,mean_val_loss[4,:],'o--',label = 'batch size = 50 - iteration = 5')
plt.title('Evolution of the validation loss with different batch sizes ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### OVERLAP TEST

In [ ]:
overlap_array = [None,42,36,24,12]
val_loss_overlap = np.zeros((5,50))

In [ ]:
for i in range(5):

    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=overlap_array[i]) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)
    
    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=5,
                                                    augment=True,
                                                    augment_flip=True)


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=2,
                                                    augment=False,
                                                    augment_flip=False)

    model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
    bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
    bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
    bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
    bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
    bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

    for i in range(14,24):
        if i ==14 :
            l = model.layers[i](bottle_6)
        elif i == 15 :
            l = model.layers[i]([model.layers[6].output,l])
        else :
            l= model.layers[i](l)

    bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

    i = 0
    for layer in bottle_model.layers:
        if i > 7 and i < 14:
            layer.trainable = True
        else:
            layer.trainable = False
        i = i+1

    bottle_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(bottle_model.optimizer.learning_rate,0.001)


    #fit the bottle model
    history = train_model(model=bottle_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["overlap_test",str(i)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=50,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_overlap[i,:] = history.history['val_loss']
    
np.save(main_dir + "parameters_analysis/overlap.npy",val_loss_overlap)

In [ ]:
val_loss_overlap = np.load(main_dir+"parameters_analysis/overlap.npy")
epochs = range(20,51,5)
mean_val_loss = np.zeros((5,7))
for i in range(7):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_overlap[0,15+5*i:15+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_overlap[1,15+5*i:15+5*i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_overlap[2,15+5*i:15+5*i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_overlap[3,15+5*i:15+5*i+5]))
    mean_val_loss[4,i]=np.abs(np.mean(val_loss_overlap[4,15+5*i:15+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'No overlap (= 48)')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'overlap of 42')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'overlap of 36')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'overlap of 24')
plt.plot(epochs,mean_val_loss[4,:],'o--',label = 'overlap of 12')
plt.title('Evolution of the validation loss with different overlap sizes ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### LEARNING RATE TEST

In [ ]:
learning_array = [0.1,0.01,0.001,0.0001]
val_loss_learning = np.zeros((4,100))

In [ ]:
X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=12) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)


for i in range(4):



    model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
    bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
    bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
    bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
    bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
    bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

    for i in range(14,24):
        if i ==14 :
            l = model.layers[i](bottle_6)
        elif i == 15 :
            l = model.layers[i]([model.layers[6].output,l])
        else :
            l= model.layers[i](l)

    bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

    i = 0
    for layer in bottle_model.layers:
        if i > 7 and i < 14:
            layer.trainable = True
        else:
            layer.trainable = False
        i = i+1

    bottle_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(bottle_model.optimizer.learning_rate,0.001)


    #fit the bottle model
    history = train_model(model=bottle_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["learning_rate_test",str(learning_array[i])]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_learning[i,:] = history.history['val_loss']
    
np.save(main_dir+"parameters_analysis/learning_rate.npy",val_loss_learning)

In [ ]:
val_loss_learning = np.load(main_dir+"parameters_analysis/learning_rate.npy")
epochs = range(25,101,5)
mean_val_loss = np.zeros((4,16))
for i in range(16):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_learning[0,20+5*i:20+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_learning[1,20+5*i:20+5*i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_learning[2,20+5*i:20+5*i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_learning[3,20+5*i:20+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'learning rate = 0.1')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'learning rate = 0.01')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'learning rate = 0.001')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'learning rate = 0.0001')

plt.title('Evolution of the validation loss with different learning rates ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### UNFREEZING TEST

We test here which layers we should kept frozen during unfreezing phase.

In [ ]:

val_loss_unfreeze = np.zeros((3,50))

In [ ]:


X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=12) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)

model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

for i in range(14,24):
    if i ==14 :
        l = model.layers[i](bottle_6)
    elif i == 15 :
        l = model.layers[i]([model.layers[6].output,l])
    else :
        l= model.layers[i](l)
        
bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

i = 0
for layer in bottle_model.layers:
    if i > 7 and i < 14:
        layer.trainable = True
    else:
        layer.trainable = False
    i = i+1
    
bottle_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(bottle_model.optimizer.learning_rate,0.001)


#fit the long model
history = train_model(model=bottle_model,
        path2save=config["path2save_transferlearning"],
        model_name='pretraining_unfreeze_test',
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=50,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )

for i in range(3):
    pretrained_model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/transferLearned_models/best_pretraining_unfreeze_test.h5')
    for layer in pretrained_model.layers:
        layer.trainable = True
    if i==1 : #freeze contraction
        for layer in pretrained_model.layers[:7]:
            layer.trainable = False
    elif i==2 : #freeze expansion
        for layer in pretrained_model.layers[14:]:
            layer.trainable = False
    
    pretrained_model.compile(optimizer = optimizer_class_name, loss=loss)
    K.set_value(pretrained_model.optimizer.learning_rate,0.00001)
    print(pretrained_model.summary())
    
    history = train_model(model=pretrained_model,
        path2save=config["path2save_finetuned"],
        model_name=_.join(['unfreeze_test_depth',str(i)]),
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=50,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )
    val_loss_unfreeze[i,:] = history.history['val_loss']
    



    
np.save(main_dir+"parameters_analysis/unfreeze_2.npy",val_loss_unfreeze)

In [ ]:
val_loss_unfreeze = np.load(main_dir+"parameters_analysis/unfreeze_2.npy")
epochs = range(15,51,5)
mean_val_loss = np.zeros((3,8))
for i in range(8):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_unfreeze[0,10+5*i:10+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_unfreeze[1,10+5*i:10+5*i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_unfreeze[2,10+5*i:10+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'No frozen layers')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'Contraction section frozen')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'Expansion section frozen')

plt.title('Evolution of the validation loss for different unfreezing possibilities')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### UNFREEZING --> LEARNING RATE TEST

In [ ]:
learning_array = [0.0001,0.00001,0.000001]
val_loss_learning = np.zeros((4,100))

In [ ]:
X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=12) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)

model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

for i in range(14,24):
    if i ==14 :
        l = model.layers[i](bottle_6)
    elif i == 15 :
        l = model.layers[i]([model.layers[6].output,l])
    else :
        l= model.layers[i](l)
        
bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

i = 0
for layer in bottle_model.layers:
    if i > 7 and i < 14:
        layer.trainable = True
    else:
        layer.trainable = False
    i = i+1
    
bottle_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(bottle_model.optimizer.learning_rate,0.001)


#fit the bottle model
history = train_model(model=bottle_model,
        path2save=config["path2save_transferlearning"],
        model_name='pretraining_unfreeze_test',
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=100,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )
val_loss_learning[0,:] = history.history['val_loss']





for i in range(3):
    pretrained_model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/transferLearned_models/best_pretraining_unfreeze_test.h5')
    for layer in pretrained_model.layers:
        layer.trainable = True
    
    pretrained_model.compile(optimizer = optimizer_class_name, loss=loss)
    K.set_value(pretrained_model.optimizer.learning_rate,learning_array[i])
    
    history = train_model(model=pretrained_model,
        path2save=config["path2save_finetuned"],
        model_name=_.join(['unfreeze_test_learning',str(i)]),
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=100,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )
    val_loss_learning[i+1,:] = history.history['val_loss']
    



    
np.save(main_dir+"parameters_analysis/unfreeze_learning_rate.npy",val_loss_learning)

In [ ]:
val_loss_learning = np.load(main_dir+"parameters_analysis/unfreeze_learning_rate.npy")
epochs = range(25,101,5)
mean_val_loss = np.zeros((4,16))
for i in range(16):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_learning[0,20+5*i:20+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_learning[1,20+5*i:20+5*i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_learning[2,20+5*i:20+5*i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_learning[3,20+5*i:20+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'First training results (before unfreezing)')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'learning rate = $10^{-4}$')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'learning rate = $10^{-5}$')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'learning rate = $10^{-6}$')

plt.title('Evolution of the validation loss with different learning rates during unfreezing phase ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### DATA AUGMENTATION TEST

In [ ]:
augment_array = [False,True]
flip_array = [False,True]
val_loss_augment = np.zeros((2,70))
train_loss_augment = np.zeros((2,70))

In [ ]:


for i in range(2):
    
    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=16) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)

    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=5,
                                                    augment=augment_array[i],
                                                    augment_flip=flip_array[i])


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=2,
                                                    augment=False,
                                                    augment_flip=False)

    model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
    bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
    bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
    bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
    bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
    bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

    for i in range(14,24):
        if i ==14 :
            l = model.layers[i](bottle_6)
        elif i == 15 :
            l = model.layers[i]([model.layers[6].output,l])
        else :
            l= model.layers[i](l)

    bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

    i = 0
    for layer in bottle_model.layers:
        if i > 7 and i < 14:
            layer.trainable = True
        else:
            layer.trainable = False
        i = i+1

    bottle_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(bottle_model.optimizer.learning_rate,0.001)


    #fit the bottle model
    history = train_model(model=bottle_model,
            path2save=config["path2save_retraining"],
            model_name='_'.join(["augment_test",str(i)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=70,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_augment[i,:] = history.history['val_loss']
    train_loss_augment[i,:] = history.history['loss']
    
np.save(main_dir+"parameters_analysis/augment_val.npy",val_loss_augment)
np.save(main_dir+"parameters_analysis/augment_train.npy",train_loss_augment)

In [ ]:
val_loss_augment = np.load(main_dir+"parameters_analysis/augment_val.npy")
loss_augment = np.load(main_dir+"parameters_analysis/augment_train.npy")
epochs = range(20,71,5)
mean_val_loss = np.zeros((2,11))
mean_loss = np.zeros((2,11))
for i in range(11):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_augment[0,15+5*i:15+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_augment[1,15+5*i:15+5*i+5]))
    mean_loss[0,i]=np.abs(np.mean(loss_augment[0,15+5*i:15+5*i+5]))
    mean_loss[1,i]=np.abs(np.mean(loss_augment[1,15+5*i:15+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'validation loss without data augmentation')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'validation loss with data augmentation')
plt.plot(epochs,mean_loss[0,:],'o--',label = 'training loss without data augmentation')
plt.plot(epochs,mean_loss[1,:],'o--',label = 'training loss with data augmentation')

plt.title('Evolution of the validation and train loss with and without data augmentation')
# show a legend on the plot
plt.ylabel('Mean of the validation/train loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### SECOND OVERLAP TEST

In [ ]:
overlap_array = [12,6,3]
val_loss_overlap = np.zeros((3,50))
loss_overlap = np.zeros((3,50))

In [ ]:
for i in range(3):

    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=overlap_array[i]) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)
    
    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=5,
                                                    augment=True,
                                                    augment_flip=True)


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=2,
                                                    augment=False,
                                                    augment_flip=False)

    model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
    bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
    bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
    bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
    bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
    bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

    for i in range(14,24):
        if i ==14 :
            l = model.layers[i](bottle_6)
        elif i == 15 :
            l = model.layers[i]([model.layers[6].output,l])
        else :
            l= model.layers[i](l)

    bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

    i = 0
    for layer in bottle_model.layers:
        if i > 7 and i < 14:
            layer.trainable = True
        else:
            layer.trainable = False
        i = i+1

    bottle_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(bottle_model.optimizer.learning_rate,0.001)


    #fit the bottle model
    history = train_model(model=bottle_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["second_overlap_test",str(i)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=50,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_overlap[i,:] = history.history['val_loss']
    loss_overlap[i,:] = history.history['loss']
    
np.save(main_dir+"parameters_analysis/second_overlap.npy",val_loss_overlap)
np.save(main_dir+"parameters_analysis/second_overlap_train.npy",loss_overlap)

In [ ]:
val_loss_overlap = np.load(main_dir+"parameters_analysis/second_overlap.npy")
epochs = range(20,51,5)
mean_val_loss = np.zeros((3,7))
for i in range(7):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_overlap[0,15+5*i:15+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_overlap[1,15+5*i:15+5*i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_overlap[2,15+5*i:15+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'overlap of 12')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'overlap of 6')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'overlap of 3')

plt.title('Evolution of the validation loss with different overlap sizes ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### IMPACT OF THE NETWORK SURGERY

We observe here the interest of the network surgery in our fine-tuning framework.

In [ ]:
val_loss = np.zeros((2,150))

In [ ]:
X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=6) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=1,
                                                augment=False,
                                                augment_flip=False)

model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
bottle_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[7].output)
bottle_2 = keras.layers.BatchNormalization(name = 'bottle_batch_1',axis=1)(bottle_1)
bottle_3 = keras.layers.Activation('relu',name = 'bottle_activation_1')(bottle_2)
bottle_4 = keras.layers.Conv3D(96,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'bottle_conv3D_2',padding = 'same',kernel_initializer = init)(bottle_3)
bottle_5 = keras.layers.BatchNormalization(name = 'bottle_batch_2',axis = 1)(bottle_4)
bottle_6 = keras.layers.Activation('relu',name = 'bottle_activation_2')(bottle_5)

for i in range(14,24):
    if i ==14 :
        l = model.layers[i](bottle_6)
    elif i == 15 :
        l = model.layers[i]([model.layers[6].output,l])
    else :
        l= model.layers[i](l)
        
bottle_model = keras.models.Model(model.input,l,name = "bottle_model")

i = 0
for layer in bottle_model.layers:
    if i > 7 and i < 14:
        layer.trainable = True
    else:
        layer.trainable = False
    i = i+1
    
bottle_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(bottle_model.optimizer.learning_rate,0.001)


#fit the bottle model
history = train_model(model=bottle_model,
        path2save=config["path2save_transferlearning"],
        model_name='final_test',
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=75,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )

val_loss[0,:75] = history.history['val_loss']

for layer in bottle_model.layers:
    layer.trainable = True
    
for layer in bottle_model.layers[:7]:
    layer.trainable = False

bottle_model.compile(optimizer = optimizer_class_name, loss=loss)
K.set_value(bottle_model.optimizer.learning_rate,0.00001)

history = train_model(model=bottle_model,
    path2save=config["path2save_finetuned"],
    model_name="final_test",
    training_generator=train_generator,
    validation_generator=validation_generator,
    steps_per_epoch=nb_train_steps,
    validation_steps=nb_valid_steps,
    n_epochs=75,
    learning_rate_drop=0.5,
    learning_rate_patience=10
   )
val_loss[0,75:] = history.history['val_loss']






model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')
for layer in model.layers:
    layer.trainable = True
    
for layer in model.layers[:7]:
    layer.trainable = False


model.compile(optimizer = optimizer_class_name, loss=loss)
K.set_value(model.optimizer.learning_rate,0.00001)
print(model.summary())

history = train_model(model=model,
    path2save=config["path2save_retrained"],
    model_name="final_test",
    training_generator=train_generator,
    validation_generator=validation_generator,
    steps_per_epoch=nb_train_steps,
    validation_steps=nb_valid_steps,
    n_epochs=150,
    learning_rate_drop=0.5,
    learning_rate_patience=10
   )
val_loss[1,:] = history.history['val_loss']

np.save(main_dir+"parameters_analysis/final_test.npy",val_loss)

In [ ]:
val_loss = np.load(main_dir+"parameters_analysis/final_test.npy")
epochs = range(30,151,5)
mean_val_loss = np.zeros((2,25))
for i in range(25):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss[0,25+5*i:25+5*i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss[1,25+5*i:25+5*i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'Network surgery')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'No  network surgery')

plt.title('Evolution of the validation loss with or without network surgery ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()